In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [3]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [4]:
df = pd.read_csv('./data/musinsa_240119.csv', index_col=0)
df.tail()

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
8040,스포츠/용품,아우터,나일론 아노락 자켓 - 블랙 / 2033B427-001,2033B427-001,0,1,0,0.0,"('24~28세', 0)",2100.0,1275,87200.0,20,4.0,4.9,222,300.0
8041,아우터,패딩 베스트,라이트 덕다운 패딩 베스트 [브라운],TMPD512340BR,0,0,0,NaN,"('~18세', 0)",1200.0,112,187000.0,0,5.0,5.0,2,NaN
8042,아우터,패딩 베스트,[MENS] 리버서블 패딩 조끼(10 color),HBA-VT-M201,0,0,0,0.0,"('34~39세', 0)",1400.0,198,38400.0,60,5.0,4.8,33,100.0
8043,아우터,숏패딩/숏헤비 아우터,숏 미니멀 덕다운 패딩 블랙,3001,0,1,0,0.0,"('29~33세', 0)",3300.0,2028,238400.0,20,7.0,4.8,87,100.0
8044,아우터,아노락 재킷,Pocket Tidy Hood Anorak H7 Navy/Green,409,1,0,2,1.0,"('34~39세', 0)",400.0,368,53000.0,40,5.0,5.0,22,50.0


In [5]:
df.isnull().sum()

major_category        0
middle_category       0
name                  0
number                0
limit                 0
exclusive             0
target_gender         0
buy_gender         2275
buy_age               0
view                388
like                  0
price                 0
discount_rate         0
delivery_date        11
rating              995
review                0
buy                2351
dtype: int64



---



## 중복행 제거

In [6]:
df = df.drop_duplicates(keep='first', ignore_index=True)
df.tail()

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
7748,스포츠/용품,아우터,나일론 아노락 자켓 - 블랙 / 2033B427-001,2033B427-001,0,1,0,0.0,"('24~28세', 0)",2100.0,1275,87200.0,20,4.0,4.9,222,300.0
7749,아우터,패딩 베스트,라이트 덕다운 패딩 베스트 [브라운],TMPD512340BR,0,0,0,NaN,"('~18세', 0)",1200.0,112,187000.0,0,5.0,5.0,2,NaN
7750,아우터,패딩 베스트,[MENS] 리버서블 패딩 조끼(10 color),HBA-VT-M201,0,0,0,0.0,"('34~39세', 0)",1400.0,198,38400.0,60,5.0,4.8,33,100.0
7751,아우터,숏패딩/숏헤비 아우터,숏 미니멀 덕다운 패딩 블랙,3001,0,1,0,0.0,"('29~33세', 0)",3300.0,2028,238400.0,20,7.0,4.8,87,100.0
7752,아우터,아노락 재킷,Pocket Tidy Hood Anorak H7 Navy/Green,409,1,0,2,1.0,"('34~39세', 0)",400.0,368,53000.0,40,5.0,5.0,22,50.0


In [7]:
df.isnull().sum()

major_category        0
middle_category       0
name                  0
number                0
limit                 0
exclusive             0
target_gender         0
buy_gender         2275
buy_age               0
view                388
like                  0
price                 0
discount_rate         0
delivery_date        11
rating              995
review                0
buy                2351
dtype: int64

## Null값 처리

### buy의 Null값 처리
- 평점이 0, 조회수가 0이어도, 조회수가 200 이하인 상품은 buy가 NaN이어도 유지함
- 정말 안 팔려서 NaN으로 수집된 아이템으로 고려
- 그 외 NaN값은 drop

-> 처리 결과 20개 정도만 남음

In [8]:
bool1 = df.rating.isnull()
bool2 = df.review == 0
bool3 = df.view < 200
bool4 = df.buy.isnull()
df.loc[(bool1 & bool2)&  bool3 &bool4, 'buy'] = 0

In [9]:
df.dropna(subset=['buy'], inplace=True)

In [10]:
df.isnull().sum()

major_category      0
middle_category     0
name                0
number              0
limit               0
exclusive           0
target_gender       0
buy_gender         63
buy_age             0
view                5
like                0
price               0
discount_rate       0
delivery_date       4
rating             91
review              0
buy                 0
dtype: int64

### buy_gender Null값 처리
- target_gender값과 동일하게 입력

In [11]:
df.loc[df['buy_gender'].isnull() == True, 'buy_gender'] = df.loc[df['buy_gender'].isnull() == True, 'target_gender']

### 나머지 결측값 처리를 위해 데이터 통계량 확인

In [12]:
df.describe()

,limit,exclusive,target_gender,buy_gender,view,like,price,discount_rate,delivery_date,rating,review,buy
count,5465.000000,5465.000000,5465.000000,5465.000000,5460.000000,5465.000000,5.465000e+03,5465.000000,5461.000000,5374.000000,5465.000000,5465.000000
mean,0.257274,0.048856,0.784812,0.420311,5887.133700,4089.295517,8.633846e+04,29.480695,5.613441,4.800521,605.925343,1174.592864
std,0.437171,0.215587,0.825222,0.496611,17049.251058,11807.340556,9.016400e+04,21.831941,2.766194,0.187413,3463.436133,5905.384029
min,0.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000e+00,0.000000,3.000000,2.300000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1400.000000,409.000000,3.890000e+04,10.000000,5.000000,4.700000,16.000000,50.000000
50%,0.000000,0.000000,1.000000,0.000000,2400.000000,1028.000000,5.940000e+04,30.000000,5.000000,4.800000,44.000000,100.000000
75%,1.000000,0.000000,2.000000,1.000000,4400.000000,2733.000000,1.042500e+05,44.000000,6.000000,4.900000,178.000000,450.000000
max,1.000000,1.000000,2.000000,2.000000,560000.000000,243639.000000,2.450000e+06,90.000000,60.000000,5.000000,107179.000000,189000.000000


### delivery_date Null값 처리
- max값을 보아 이상치가 있는 것을 확인, 중앙값인 5일로 입력

In [13]:
df.loc[df['delivery_date'].isnull() == True, 'delivery_date'] = 5.0

### rating의 Null값 처리

In [14]:
notzerobuy_df = df.loc[(df['buy'] != 0) & (df['rating'].isnull())]
notzerobuy_df

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
138,상의,후드 티셔츠,메테오샤워 후드 오트밀 MJHD7698,5004339106,0,0,0,1.0,"('19~23세', 0)",2300.0,442,43900.0,26,5.0,NaN,0,50.0
174,상의,니트/스웨터,하프 집업 오버핏 케이블 니트 - 6 COLOR,JT68,0,0,2,0.0,"('29~33세', 0)",5700.0,254,34800.0,78,6.0,NaN,0,50.0
803,상의,긴소매 티셔츠,울U넥 티,78,0,0,1,1.0,"('29~33세', 0)",150.0,92,48000.0,0,5.0,NaN,0,50.0
1144,바지,트레이닝/조거 팬츠,JORDAN PIGMENT TRACKSUIT PANTS (WHITE),GY23CPT002WH,0,0,1,1.0,"('24~28세', 0)",2100.0,394,108000.0,0,5.0,NaN,0,50.0
1248,바지,기타 바지,[기모] 나일론 스트링 카고 팬츠 그레이,PY-23176 GY,1,0,0,0.0,"('29~33세', 0)",2000.0,320,31200.0,50,5.0,NaN,0,50.0
1681,상의,맨투맨/스웨트셔츠,CROP SWEAT BOLERO WHITE MELANGE,CA22DCD001WM,0,0,1,1.0,"('19~23세', 0)",1200.0,2773,42500.0,35,21.0,NaN,115,350.0
1682,상의,니트/스웨터,Pakrat Wool Knit Pullover Oatmeal,MDKNDC001OT,1,0,0,1.0,"('19~23세', 0)",2000.0,397,63200.0,20,5.0,NaN,0,50.0
1754,상의,맨투맨/스웨트셔츠,[세트] 돌핀 자수 엣지 기모 스웻셔츠 조거 트레이닝 팬츠 블랙,134322722BK,0,0,0,1.0,"('40세~', 0)",1400.0,117,79900.0,53,5.0,NaN,0,50.0
1849,상의,후드 티셔츠,HOME PAINTING PRINTED HOODIE (WHITE),11698,0,0,2,1.0,"('24~28세', 0)",3900.0,616,132300.0,10,7.0,NaN,0,50.0
1850,상의,맨투맨/스웨트셔츠,UNISEX DUNST LOGO CREASE SWEATSHIRT FRENCH NAV...,UDTS4A103N2,0,0,1,1.0,"('24~28세', 0)",950.0,1683,65550.0,5,7.0,NaN,110,300.0


In [15]:
notzerobuy_df.isnull().sum()

major_category      0
middle_category     0
name                0
number              0
limit               0
exclusive           0
target_gender       0
buy_gender          0
buy_age             0
view                0
like                0
price               0
discount_rate       0
delivery_date       0
rating             28
review              0
buy                 0
dtype: int64

### rating의 Null값 처리
  - buy값이 0이 아니면서 rating이 0인 22개 -> 4.8
  - buy값이 0이면서 rating이 0인 27개 -> 0

In [16]:
df.loc[df['rating'].isnull() == True, 'rating'] = 0.0

for i in notzerobuy_df.index:
  df.loc[i, 'rating'] = 4.8

### view의 Null값 처리
- RandomForestRegressor 모델을 이용한 예측값 사용
- 여러가지 파라미터 조정 결과
  - Best r2_Score: 0.47624328080184863
  - Best trial parameter {'n_estimators': 281, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.8499303073699949}

In [17]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import r2_score

# # DataFrame에서 'view'가 결측치인 행을 제외한 데이터로 모델을 학습시키기 위해 두 개의 서브셋을 생성
# train_data = df.dropna(subset=['view'])
# predict_data = df[df['view'].isnull()]

# # 특성과 타겟을 정의
# features = ['like', 'price', 'discount_rate', 'rating', 'review', 'buy']
# target = 'view'

# # 학습 데이터셋과 타겟을 설정
# X = train_data[features]
# y = train_data[target]

# #train, test 데이터셋 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# params = {'n_estimators': 281, 'max_depth': 20, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 0.8499303073699949}
# model = RandomForestRegressor(**params)

# # 학습 데이터셋을 이용하여 모델을 학습
# model.fit(X_train, y_train)

# # 예측 데이터셋을 이용하여 'view'의 결측치를 예측
# predicted_views = model.predict(X_test)

# r2_score(y_test, predicted_views)

In [18]:
# pred = model.predict(df.loc[df.view.isnull(), ['like', 'price', 'discount_rate', 'rating', 'review', 'buy']])
# df.loc[df['view'].isnull(), 'view'] = pred

In [19]:
df.isnull().sum()

major_category     0
middle_category    0
name               0
number             0
limit              0
exclusive          0
target_gender      0
buy_gender         0
buy_age            0
view               5
like               0
price              0
discount_rate      0
delivery_date      0
rating             0
review             0
buy                0
dtype: int64

In [20]:
import copy
# 결측치 처리 완료 df 백업
backup = df.copy()

In [21]:
df = backup.copy()

---

In [22]:
df.reset_index(drop=True, inplace=True)

In [23]:
df

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy
0,상의,셔츠/블라우스,오버사이즈 16골 에센셜 코듀로이 셔츠 블랙,22FWGS28BK,1,0,2,0.0,"('29~33세', 0)",1200.0,493,39800.0,20,19.0,4.6,36,50.0
1,상의,니트/스웨터,TD5-SW01 램스울 라운드 니트-만다린오렌지,TD5-SW01_MAO,1,0,0,0.0,"('29~33세', 0)",4900.0,650,53550.0,10,5.0,4.9,61,200.0
2,상의,후드 티셔츠,[SET UP] PPP 플라워 후드 셋업_멜란지 그레이,KBCS1TH003MGKBCS1PL003MGKBCS1PS003MG,1,0,2,1.0,"('19~23세', 0)",3600.0,2536,49900.0,49,5.0,4.8,41,150.0
3,상의,후드 티셔츠,TIE DYE 후드티 Olive Green,CTTZPHD01UG4,0,0,0,0.0,"('40세~', 0)",1000.0,569,19800.0,80,5.0,4.8,62,150.0
4,상의,긴소매 티셔츠,essential turtleneck logo top - pink,BT23WTS002PIKF,1,0,1,1.0,"('24~28세', 0)",11000.0,2561,43000.0,0,28.0,4.8,34,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5460,아우터,숏패딩/숏헤비 아우터,다운 숏 푸퍼-블랙,P22FWOT001,0,0,2,0.0,"('19~23세', 0)",2900.0,771,139000.0,43,5.0,4.9,17,50.0
5461,스포츠/용품,아우터,나일론 아노락 자켓 - 블랙 / 2033B427-001,2033B427-001,0,1,0,0.0,"('24~28세', 0)",2100.0,1275,87200.0,20,4.0,4.9,222,300.0
5462,아우터,패딩 베스트,[MENS] 리버서블 패딩 조끼(10 color),HBA-VT-M201,0,0,0,0.0,"('34~39세', 0)",1400.0,198,38400.0,60,5.0,4.8,33,100.0
5463,아우터,숏패딩/숏헤비 아우터,숏 미니멀 덕다운 패딩 블랙,3001,0,1,0,0.0,"('29~33세', 0)",3300.0,2028,238400.0,20,7.0,4.8,87,100.0


## buy_age 변경
- 동률인 경우가 있기 때문에 buy_age1, buy_age2로 컬럼을 나눔
- 범위를 0,1,2,3,4,5,6으로 구분지어주었음

In [24]:
import re

def age1(row):
  numbers = [int(num) for num in re.findall(r'\d+', row)]
  return numbers[0]

df['buy_age1'] = df['buy_age'].apply(age1)

In [25]:
def age2(row):
  numbers = [int(num) for num in re.findall(r'\d+', row)]
  return numbers[1]

df['buy_age2'] = df['buy_age'].apply(age2)

In [26]:
df.buy_age1.unique()

array([29, 19, 40, 24, 18, 34,  0])

In [27]:
def age_range(row):
  if row == 40:
    return 6
  elif row == 18:
    return 1
  elif row == 19:
    return 2
  elif row == 24:
    return 3
  elif row == 29:
    return 4
  elif row == 34:
    return 5
  else: return 0

In [28]:
df['buy_age1'] = df['buy_age1'].apply(age_range)
df['buy_age2'] = df['buy_age2'].apply(age_range)

In [29]:
df.head()

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy,buy_age1,buy_age2
0,상의,셔츠/블라우스,오버사이즈 16골 에센셜 코듀로이 셔츠 블랙,22FWGS28BK,1,0,2,0.0,"('29~33세', 0)",1200.0,493,39800.0,20,19.0,4.6,36,50.0,4,0
1,상의,니트/스웨터,TD5-SW01 램스울 라운드 니트-만다린오렌지,TD5-SW01_MAO,1,0,0,0.0,"('29~33세', 0)",4900.0,650,53550.0,10,5.0,4.9,61,200.0,4,0
2,상의,후드 티셔츠,[SET UP] PPP 플라워 후드 셋업_멜란지 그레이,KBCS1TH003MGKBCS1PL003MGKBCS1PS003MG,1,0,2,1.0,"('19~23세', 0)",3600.0,2536,49900.0,49,5.0,4.8,41,150.0,2,0
3,상의,후드 티셔츠,TIE DYE 후드티 Olive Green,CTTZPHD01UG4,0,0,0,0.0,"('40세~', 0)",1000.0,569,19800.0,80,5.0,4.8,62,150.0,6,0
4,상의,긴소매 티셔츠,essential turtleneck logo top - pink,BT23WTS002PIKF,1,0,1,1.0,"('24~28세', 0)",11000.0,2561,43000.0,0,28.0,4.8,34,100.0,3,0


### 상의-스포츠/용품 위치 바꾸기

In [30]:
df.loc[df.middle_category == '상의', 'major_category'] = '상의'

In [31]:
df.loc[df.middle_category == '상의','middle_category'] = '스포츠/용품'

In [32]:
df.head()

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,like,price,discount_rate,delivery_date,rating,review,buy,buy_age1,buy_age2
0,상의,셔츠/블라우스,오버사이즈 16골 에센셜 코듀로이 셔츠 블랙,22FWGS28BK,1,0,2,0.0,"('29~33세', 0)",1200.0,493,39800.0,20,19.0,4.6,36,50.0,4,0
1,상의,니트/스웨터,TD5-SW01 램스울 라운드 니트-만다린오렌지,TD5-SW01_MAO,1,0,0,0.0,"('29~33세', 0)",4900.0,650,53550.0,10,5.0,4.9,61,200.0,4,0
2,상의,후드 티셔츠,[SET UP] PPP 플라워 후드 셋업_멜란지 그레이,KBCS1TH003MGKBCS1PL003MGKBCS1PS003MG,1,0,2,1.0,"('19~23세', 0)",3600.0,2536,49900.0,49,5.0,4.8,41,150.0,2,0
3,상의,후드 티셔츠,TIE DYE 후드티 Olive Green,CTTZPHD01UG4,0,0,0,0.0,"('40세~', 0)",1000.0,569,19800.0,80,5.0,4.8,62,150.0,6,0
4,상의,긴소매 티셔츠,essential turtleneck logo top - pink,BT23WTS002PIKF,1,0,1,1.0,"('24~28세', 0)",11000.0,2561,43000.0,0,28.0,4.8,34,100.0,3,0


In [33]:
df = pd.read_csv('./data/데이터 전처리완성20240121.csv', index_col=0)
df.tail()

,major_category,middle_category,name,number,limit,exclusive,target_gender,buy_gender,buy_age,view,...,price,discount_rate,delivery_date,rating,review,buy,buy_age1,buy_age2,item_id,category
7086,아우터,환절기 코트,W MAC COAT camel,0MG22SCT101,0,0,1,1.0,"('40세~', 0)",9000.0,...,77700.0,70,7.0,5.0,6,50.0,6,0,5073,40
7129,아우터,환절기 코트,military balmacaan coat black,UB-3347,1,0,0,0.0,"('24~28세', 0)",2200.0,...,93210.0,61,5.0,4.9,18,50.0,3,0,5106,40
7256,아우터,환절기 코트,클래식 오버핏 베이직 더블 트렌치 코트 자켓,23S,1,0,1,1.0,"('19~23세', 0)",1100.0,...,47100.0,40,6.0,4.5,63,200.0,2,0,5208,40
7334,아우터,환절기 코트,CHEST DOUBLE POCKET COAT_BEIGE,20BAFWOCTU001BE,0,0,0,0.0,"('40세~', 0)",2200.0,...,39800.0,80,5.0,5.0,15,50.0,6,0,5248,40
7628,아우터,환절기 코트,제이드 롱 레더 코트 (블랙),NRCO3C001BK,0,0,1,1.0,"('29~33세', 0)",1700.0,...,239000.0,0,5.0,4.6,8,50.0,4,0,5405,40


---